# Capstone project of the Coursera

In [1]:
import requests
import pandas as pd
import numpy as np
import random

from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

import folium

I have used the fourquare RESTful APIs to collect the information about the location
1. App settings -> Redirect URL -> Web Addresses-> https://www.google.com

2. To get access token:

3. Replace it with the client_id https://foursquare.com/oauth2/authenticate?client_id=YOUR_CLIENT_ID&response_type=code&redirect_uri=https://www.google.com

4. It should look like https://www.google.com/?code=CODE take the code part

5. Replace all client_id, sceret and code to get the access token https://foursquare.com/oauth2/access_token?client_id=YOUR_CLIENT_ID&client_secret=YOUR_CLIENT_SECRET&grant_type=authorization_code&redirect_uri=https://www.google.com&code=CODE. 

In [66]:
CLIENT_ID = 'NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N' # your Foursquare ID
CLIENT_SECRET = 'WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ' # your Foursquare Secret
ACCESS_TOKEN = 'AWCDFCLJBBPUKSG2FX5TTMHSRG2OEIPTEQ5P3CH0DBSENWWX' # your FourSquare Access Token
VERSION = '20210201'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N
CLIENT_SECRET:WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ


In [3]:
address = '102 North End Ave, New York, NY'

geolocator= Nominatim(user_agent="foursquare_agent")
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7149555 -74.0153365


# 1. Search for a specific venue category

    https://api.foursquare.com/v2/venues/search?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&query=QUERY&radius=RADIUS&limit=LIMIT

In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [5]:
url="https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&query={}&limit={}".format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,radius,search_query, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&ll=40.7149555,-74.0153365&v=20210201&radius=500&query=Italian&limit=30'

In [6]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601acfd144ad3d0ca7344381'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 58,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-16123698

In [7]:
venues=results['response']['venues']

df=pd.json_normalize(venues)
df

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1612369873,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1612369873,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN
2,3fd66200f964a520f4e41ee3,Ecco,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1612369873,False,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway


In [10]:
filtered_columns=['name','categories']+[col for col in df.columns if col.startswith ('location.')]+['id']
df_filtered=df.loc[:,filtered_columns]

df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]
 
def get_cat(row):
    cat = row['categories']
    if len(cat)==0:
        return None
    else:
        return cat[0]['name']

df_filtered['categories']=df.apply(get_cat,axis=1)

df_filtered
                                                                                                

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",58,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",NaN,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",469,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",NaN,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,40.715337,-74.008848,"[{'label': 'display', 'lat': 40.71533713859952...",549,10007,US,New York,NY,United States,[124 Chambers St (btwn Church St & W Broadway)...,btwn Church St & W Broadway,3fd66200f964a520f4e41ee3


In [11]:
venues_map=folium.Map(location=[latitude,longitude],zoom_start=13)

folium.CircleMarker(
    [latitude,longitude],
    radius=10,
    color='red',
    popup='Corad Hotel',
    fill=True,
    fill_color='red',
    fill_opacity=0.6
).add_to(venues_map)

for lat,lon,label in zip(df_filtered.lat,df_filtered.lng,df_filtered.categories):
    folium.CircleMarker(
    [lat,lon],
    radius=10,
    color='red',
    popup=label,
    fill=True,
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)
    

venues_map

# 2. Explore a Given Venue

    https://api.foursquare.com/v2/venues/VENUE_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION



In [12]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&v=20210201'

In [13]:
results=requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601ad04e2efe1b20da5bf4bf'},
 'response': {'venue': {'id': '4fa862b3e4b0ebff2f749f06',
   'name': "Harry's Italian Pizza Bar",
   'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
   'location': {'address': '225 Murray St',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.71521779064671,
      'lng': -74.01473940209351},
     {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
    'postalCode': '10282',
    'cc': 'US',
    'city': 'New York',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['225 Murray St',
     'New York, NY 10282',
     'United States']},
   'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
   'categories': [{'id': '4bf58dd8d48988d1ca941735',
     'name': 'Pizza Place',
     'pluralName': 'Pizza Places',
     'shortName': 'Pizza',
     'icon': {'prefix': 'https

In [14]:
try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

6.9


In [15]:
venue_id = '4f3232e219836c91c7bfde94'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results=requests.get(url).json()

try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

no results found


In [16]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

results=requests.get(url).json()

try:
    print(results['response']['venue']['rating'])
except:
    print("no results found")

7.3


In [17]:
results['response']['venue']['tips']['count']

19

In [35]:
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601ad2f687cc0c3cbb34962f'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 5,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'firstName': 'Nick',
      'lastName': 'E',
      'address': '',
      'city': '',
      'state': ''},
     'authorInteractionType': 'liked'}]}}}

In [37]:
tips = results['response']['tips']['items']

tips = results['response']['tips']['items'][0]
tips.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

In [39]:
tips_df=pd.json_normalize(tips)

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName']
tips_filtered = tips_df.loc[:,filtered_columns]

tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",5,0,5ab1cb46c9a517174651d3fe,Nick,E


# 3. Search a Foursquare user



    https://api.foursquare.com/v2/users/USER_ID?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&v=VERSION


In [69]:
User_ID = '48454224'

url='https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}'.format(User_ID, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/users/48454224/tips?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&oauth_token=AWCDFCLJBBPUKSG2FX5TTMHSRG2OEIPTEQ5P3CH0DBSENWWX&v=20210201'

In [71]:
results=requests.get(url).json()
results



{'meta': {'code': 200, 'requestId': '601ae6c6d8f1871adf88bf42'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 1,
   'items': [{'id': '518f4181498ed1b8ddf4d733',
     'createdAt': 1368342913,
     'text': 'El lugar esta muy bueno, pero el DJ nada más no!!!!!!',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/518f4181498ed1b8ddf4d733',
     'likes': {'count': 1,
      'groups': [{'type': 'others', 'count': 1, 'items': []}],
      'summary': '1 like'},
     'like': False,
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'venue': {'id': '50779500e4b0112fc267c1da',
      'name': 'Tres Bar',
      'location': {'lat': 20.17693758560229,
       'lng': -98.04832423694748,
       'labeledLatLngs': [{'label': 'display',
         'lat': 20.17693758560229,
         'lng': -98.04832423694748}],
       'cc': 'MX',
       'country': 'México',
       'formattedAddr

In [88]:
a=results['response']['tips']['items'][0]['venue']['photos']['groups'][0]['items']
d=pd.json_normalize(a)

filtered_columns = ['user.id','user.firstName','user.lastName']
tips_filtered = d.loc[:, filtered_columns]


df=tips_filtered
df


,user.id,user.firstName,user.lastName
0,133773133,Cesar,H. Gamboa
1,30238440,Cesar,H. Gamboa
2,51713900,Marcs,Marquez Mejorada


In [89]:

g=df.loc[df['user.id'] == '133773133']
print('First Name: ' + g['user.firstName'])
print('Last Name: ' + g['user.lastName'])

0    First Name: Cesar
Name: user.firstName, dtype: object
0    Last Name: H. Gamboa
Name: user.lastName, dtype: object


In [90]:
Image(url='https://fastly.4sqi.net/img/general/540x920/133773133_ODR5Au05ENkSyu3xxfV3VOfCa0idDfm9Q4n8YbhZDRQ.jpg')

To get the venue for which many no of agreed are there for a tip

In [118]:
user_id='484542633'
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN,VERSION, limit)

results = requests.get(url).json()
tips = results['response']['tips']['items']



tips_df = pd.json_normalize(tips)

filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

tips_filtered

,text,agreeCount,disagreeCount,id
0,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
1,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
2,Burger game strong 💪,1,0,5ab575fb6bdee65f759da8c1
3,"Great burgers & fries! Also, this place is exactly what it’s like when you go to a bar in the Southwest. Source: I’m from Arizona.",2,0,5ab5575d73fe2516ad8f363b
4,Açaí bowl + peanut butter + whey protein = 💪💪💪,1,0,5ab42db53c858d64af2688a4
5,Highly underrated and way less crowded than Central Park!,3,0,5ab42c396f706a29f53ad1a8
6,Get the açaí bowl with peanut butter after your work out and thank me later 👌,1,0,5ab42aca2a7ab6333652b266
7,"When you want a burger, this should be the first thing that comes to mind. A+!",1,0,5ab42a28da5e5617d18e3a6a
8,Way less crowded than Central Park! People who live in the neighborhood rave about Carl Schurz Park.,3,0,5ab429db1ffe971b060083f5
9,The best Mexican food in the Murray Hill / Kips Bay area!,1,0,5ab3f53f8496ca57542d5549


In [115]:
tip_id = '5ab3f275dff815096a61884e' # tip id

url = 'https://api.foursquare.com/v2/tips/{}/venue?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET,VERSION) 

url


'https://api.foursquare.com/v2/tips/5ab3f275dff815096a61884e/venue?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&v=20210201'

In [120]:
result = requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '601af39390f4b77231b795c4'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 94,
   'items': [{'id': '5acbbd4eb1538e45373b07f5',
     'createdAt': 1523301710,
     'text': 'You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5acbbd4eb1538e45373b07f5',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 1,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'venue': {'id': '5431dd38498e412bc3d19e4c',
      'name': 'Insomnia Cookies',
      'location': {'address': '304 W 14th St',
       'lat': 40.74000439332591,
       'lng': -74.00308981191287,
       'labeledLatLngs': [{'label': 'display',
         'lat': 40.74000439332591,
         'lng': -74.00308981191287}],
       'postalCode': '10014',
       'cc': 'US',
       'city

In [121]:
print(result['response']['tips']['items'][0]['venue']['name'])
print(result['response']['tips']['items'][0]['venue']['location'])

Insomnia Cookies
{'address': '304 W 14th St', 'lat': 40.74000439332591, 'lng': -74.00308981191287, 'labeledLatLngs': [{'label': 'display', 'lat': 40.74000439332591, 'lng': -74.00308981191287}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['304 W 14th St', 'New York, NY 10014', 'United States']}


# 4. Explore a location

    https://api.foursquare.com/v2/venues/explore?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION&limit=LIMIT



To get the popular spots around the location

In [171]:
latitude = 40.715337
longitude = -74.008848

In [123]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=NX2GTOOXPXMNZGR2FQP4COOV3NPCYVWMVKCTCJB54CLEOL0N&client_secret=WIFEAP2OAFGGR1OGTQOHMQWHBP3ZD1ZQM2S42K0FPG3BO0MJ&ll=40.715337,-74.008848&v=20210201&radius=500&limit=30'

In [126]:
results=requests.get(url).json()
"the no of spots are:{}".format(len(results['response']['groups'][0]['items']))

'the no of spots are:30'

In [150]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5d5f24ec09484500079aee00',
  'name': 'Los Tacos No. 1',
  'location': {'address': '136 Church St',
   'lat': 40.714267,
   'lng': -74.008756,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.714267,
     'lng': -74.008756}],
   'distance': 119,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['136 Church St',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d151941735',
    'name': 'Taco Place',
    'pluralName': 'Taco Places',
    'shortName': 'Tacos',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_',
     'suffix': '.png'},
    'primary': True}],
  'delivery': {'id': '2180700',
   'url': 'https://www.seamless.com/menu/los-tacos-no-1-tribeca-136-church-st-new-york/21

In [151]:
df=pd.json_normalize(items)
d=['venue.id','venue.name','venue.categories']+[col for col in df.columns if col.startswith('venue.location.')]
df=df.loc[:,d]
df.head()

,venue.id,venue.name,venue.categories,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.location.crossStreet,venue.location.neighborhood
0,5d5f24ec09484500079aee00,Los Tacos No. 1,"[{'id': '4bf58dd8d48988d151941735', 'name': 'Taco Place', 'pluralName': 'Taco Places', 'shortName': 'Tacos', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/taco_', 'suffix': '.png'}, 'primary': True}]",136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,NaN
1,4af5d65ff964a52091fd21e3,Korin,"[{'id': '4bf58dd8d48988d1f8941735', 'name': 'Furniture / Home Store', 'pluralName': 'Furniture / Home Stores', 'shortName': 'Furniture / Home', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_', 'suffix': '.png'}, 'primary': True}]",57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Church St,Tribeca
2,4c154c9a77cea593c401d260,Takahachi Bakery,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'Bakery', 'pluralName': 'Bakeries', 'shortName': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}, 'primary': True}]",25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",at Church St,NaN
3,54148bc6498ea7bb8c05b70a,Juice Press,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]",83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",btwn Greenwich St & W Broadway,NaN
4,57ad129c498e05b086594d72,Heyday,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'Spa', 'pluralName': 'Spas', 'shortName': 'Spa', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_', 'suffix': '.png'}, 'primary': True}]",92 Reade St,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': 'entrance', 'lat': 40.715654, 'lng': -74.00782}]",100,10013,US,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",NaN,NaN


In [167]:
df.columns=[col.split('.')[-1] for col in df.columns]

def get_cat(cat):
    a=cat.categories
    if len(a)==0:
        return Null
    else:
        return cat.categories[0]['name']

df['categories']= df.apply(get_cat,axis=1)
df

,id,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood
0,5d5f24ec09484500079aee00,Los Tacos No. 1,Taco Place,136 Church St,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,NaN
1,4af5d65ff964a52091fd21e3,Korin,Furniture / Home Store,57 Warren St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Church St,Tribeca
2,4c154c9a77cea593c401d260,Takahachi Bakery,Bakery,25 Murray St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",at Church St,NaN
3,54148bc6498ea7bb8c05b70a,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",btwn Greenwich St & W Broadway,NaN
4,57ad129c498e05b086594d72,Heyday,Spa,92 Reade St,40.715726,-74.007767,"[{'label': 'display', 'lat': 40.715726, 'lng': -74.007767}, {'label': 'entrance', 'lat': 40.715654, 'lng': -74.00782}]",100,10013,US,New York,NY,United States,"[92 Reade St, New York, NY 10013, United States]",NaN,NaN
5,4a8f2f39f964a520471420e3,Takahachi,Sushi Restaurant,145 Duane St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",btwn W Broadway & Church St,NaN
6,4adcf23cf964a520cc6221e3,Chambers Street Wines,Wine Shop,148 Chambers St,40.715773,-74.009718,"[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}, {'label': 'entrance', 'lat': 40.715696, 'lng': -74.00988}]",88,10007,US,New York,NY,United States,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]",btwn West Broadway & Hudson St,NaN
7,5dc6f6a5ea8dfb00080f6faa,Lekka Burger,Burger Joint,81 Warren St,40.715246,-74.010559,"[{'label': 'display', 'lat': 40.715246, 'lng': -74.010559}]",144,10007,US,New York,NY,United States,"[81 Warren St (btw Greenwich & West Broadway), New York, NY 10007, United States]",btw Greenwich & West Broadway,NaN
8,4a6e331af964a52031d41fe3,Equinox Tribeca,Gym,54 Murray St,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",at W Broadway,NaN
9,4cd89eeb6e8b5941660c64d2,Weather Up,Cocktail Bar,159 Duane St,40.716741,-74.008666,"[{'label': 'display', 'lat': 40.71674084163369, 'lng': -74.0086664438893}, {'label': 'entrance', 'lat': 40.71685, 'lng': -74.008729}]",157,10013,US,New York,NY,United States,"[159 Duane St (btwn Hudson St. & W Broadway), New York, NY 10013, United States]",btwn Hudson St. & W Broadway,NaN


In [169]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(df.lat, df_filtered.lng, df.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map

# 5. Explore Trending Venues

    https://api.foursquare.com/v2/venues/trending?client_id=CLIENT_ID&client_secret=CLIENT_SECRET&ll=LATITUDE,LONGITUDE&v=VERSION



Now as we have explored the place now we can finf what is the trending place by:

In [174]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '601afffff6cb9000cada0351'},
 'response': {'venues': []}}

In [175]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [176]:
trending_venues_df

'No trending venues are available at the moment!'

In [1]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
venues_map

NameError: name 'results' is not defined